In [2]:
import pandas as pd

nsmc_fname = 'ratings.txt'
nsmc_total_data = pd.read_csv(nsmc_fname, sep='\t')
text_data_list = nsmc_total_data['document'].dropna().values

print(text_data_list[:4])

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ'
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.'
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.'
 '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지']


In [3]:
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Mecab; mecab=Mecab()

text_data = [['<OOV>'] + mecab.morphs(xx) for xx in tqdm(text_data_list)]

w2v_model_fname = 'nsmc_w2v_model.model'
w2v_model = Word2Vec(text_data, size=256, window=10, min_count=10, sg=1)
w2v_model.save(w2v_model_fname)

100%|██████████| 199992/199992 [00:14<00:00, 14038.06it/s]


In [4]:
w2v_model = Word2Vec.load(w2v_model_fname)

print(len(w2v_model.wv.vocab))

print(w2v_model.wv['평점'].shape)
print(w2v_model.wv.most_similar('평점', topn=4))
print(w2v_model.wv.similarity('평점', '별점'))

12066
(256,)
[('평정', 0.7146437168121338), ('별점', 0.6978053450584412), ('평', 0.6962314248085022), ('네티즌', 0.6213876605033875)]
0.6978053


In [13]:
import numpy as np

print(text_data_list[0])
print(mecab.morphs(text_data_list[0]))

test_w2v_embedd = []
for xx in mecab.morphs(text_data_list[0]):
    try:
        test_w2v_embedd.append(w2v_model.wv[xx])
    except KeyError:
        test_w2v_embedd.append(w2v_model.wv['<OOV>'])
        
print(np.asarray(test_w2v_embedd).shape)

어릴때보고 지금다시봐도 재밌어요ㅋㅋ
['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ']
(10, 256)
